In [18]:
##### EXERCICE CHAPITRE 3 #####
### PARTIE 1 ###

#1: Take the dict created in the TODO 4 in chapter I and save it in the collection "CRUD_exercise".

import pymongo
import tqdm
import numpy as np

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["TODO"]
collection = db["CRUB_exercise"]

lecun_paper = {
    "title": "Deep Learning",
    "authors": {
        "Yann LeCun": {"affiliations": ["Facebook AI Research", "New York University"]},
        "Yoshua Bengio": {"affiliations": ["Department of Computer Science and Operations Research Université de Montréal"]},
        "Geoffrey Hinton": {"affiliations": ["Google", "Department of Computer Science, University of Toronto"]}
    }
}
goodfellow_paper = {
    "title": "Generative Adversarial Nets",
    "authors": {
        "Ian Goodfellow": {"affiliations": ["Universite de Montreal"]},
        "Jean Pouget-Abadie": {"affiliations": ["Ecole Polytechnique"]},
        "Mehdi Mirza": {"affiliations": ["Université de Montréal"]},
        "Bing Xu": {"affiliations": ["Université de Montréal"]},
        "David Warde-Farley": {"affiliations": ["Université de Montréal"]},
        "Sherjil Ozair": {"affiliations": ["Indian Institute of Technology Delhi"]},
        "Aaron Courville": {"affiliations": ["Université de Montréal"]},
        "Yoshua Bengio": {"affiliations": ["CIFAR Senior Fellow"]}
    }
}
papers_dict = {
    "LeCun et al.": lecun_paper,
    "Goodfellow et al.": goodfellow_paper
}

collection.insert_one(papers_dict)


InsertOneResult(ObjectId('662a566e33b7e98b2b0ce791'), acknowledged=True)

In [2]:
#2 : Insert 3 documents with key = x and values = 1, delete one of them. Which one is deleted first ? the most recent or oldest one ? increment the value of x to 4.

for i in range(1, 4):
    collection.insert_one({"key": "x", "value": 1})

# Afficher les documents avant la suppression
print("Documents avant la suppression :")
for doc in collection.find({"key": "x"}):
    print(doc)

# Supprimer un des documents
collection.delete_one({"key": "x"})

# Afficher les documents après la suppression
print("\nDocuments après la suppression :")
for doc in collection.find({"key": "x"}):
    print(doc)


docs = list(collection.find({"key": "x"}).sort("_id", pymongo.ASCENDING))
if len(docs) == 2:
    if docs[0]["_id"] < docs[1]["_id"]:
        print("\nLe document supprimé était le plus ancien.")
    else:
        print("\nLe document supprimé était le plus récent.")
else:
    print("\nIl y a eu une erreur dans la suppression des documents.")

collection.update_many({"key": "x"}, {"$set": {"value": 4}})

print("\nDocuments après l'incrémentation de la valeur de x :")
for doc in collection.find({"key": "x"}):
    print(doc)


Documents avant la suppression :
{'_id': ObjectId('662a2773b24ef03be462b554'), 'key': 'x', 'value': 4}
{'_id': ObjectId('662a2773b24ef03be462b555'), 'key': 'x', 'value': 4}
{'_id': ObjectId('662a397733b7e98b2bfda540'), 'key': 'x', 'value': 1}
{'_id': ObjectId('662a397733b7e98b2bfda541'), 'key': 'x', 'value': 1}
{'_id': ObjectId('662a397733b7e98b2bfda542'), 'key': 'x', 'value': 1}

Documents après la suppression :
{'_id': ObjectId('662a2773b24ef03be462b555'), 'key': 'x', 'value': 4}
{'_id': ObjectId('662a397733b7e98b2bfda540'), 'key': 'x', 'value': 1}
{'_id': ObjectId('662a397733b7e98b2bfda541'), 'key': 'x', 'value': 1}
{'_id': ObjectId('662a397733b7e98b2bfda542'), 'key': 'x', 'value': 1}

Il y a eu une erreur dans la suppression des documents.

Documents après l'incrémentation de la valeur de x :
{'_id': ObjectId('662a2773b24ef03be462b555'), 'key': 'x', 'value': 4}
{'_id': ObjectId('662a397733b7e98b2bfda540'), 'key': 'x', 'value': 4}
{'_id': ObjectId('662a397733b7e98b2bfda541'), 'key':

In [3]:
# Le document supprimé est le plus ancien 

In [17]:
#3: Insert the dict created in the TODO 6 Chapter I in the example collection.

import pymongo
import json

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["tutorial"]
collection = db["example"]

# Load data from the JSON file
with open('xml_data.json', 'r') as file:
    data = json.load(file)

# Insert the data into the collection
collection.insert_one(data)





InsertOneResult(ObjectId('662a55ba33b7e98b2b0ce78f'), acknowledged=True)

In [19]:
#4 : Get documents where authors key exist in the collection "CRUD_exercise".
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["TODO"]
collection = db["CRUD_exercise"]

documents_with_authors = collection.find({"authors": {"$exists": True}})
print(documents_with_authors)


In [9]:
#5 : Change the documents where x = 4 to x = 1.

collection.update_many({"x": 4}, {"$set": {"x": 1}})


UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [15]:
#6 : Find documents where author is not_mike and set author as real_mike.
collection.update_many({"author": "not_mike"}, {"$set": {"author": "real_mike"}})


UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [11]:
#7 : Delete documents where author is real_mike
collection.delete_many({"author": "real_mike"})


DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [ ]:
### PARTIE 2 ###

In [20]:
#8 : create a collection named "CRUD_exercise_benchmark" with 500k observations, ids increment of 2 (sequence:0,2,4,6,...1M). Give a random np.array with a key named "values" and use the insert_many. Then create an index on the id and benchmark queries before and after indexing. Did the index help ?

import pymongo
client = pymongo.MongoClient('localhost')
mydb = client["TODO"]
collection = mydb["CRUD_exercise_benchmark"]

import numpy as np

values = np.random.rand(500000, 4)
docs = [{"id": i, "values": values[i//2].tolist()} for i in range(0, 1000000, 2)]

collection.insert_many(docs)

import time

# No index
start_time = time.time()
docs = collection.find({"id": 100000})
end_time = time.time()
print("Temps sans index: ", end_time - start_time)

# Index
start_time = time.time()
docs = collection.find({"id": 100000}).hint("id_1")
end_time = time.time()
print("Temps avec index: ", end_time - start_time)


Temps sans index:  0.14007258415222168
Temps avec index:  0.0


In [21]:
#9 : 
import pymongo
client = pymongo.MongoClient("mongodb://localhost:27017/")
random_db = client["random_db"]
random_collection = random_db["random_collection"]

doc = {"key": "value"}
random_collection.insert_one(doc)

tutorial_db = client["tutorial"]
tutorial_collection = tutorial_db["tutorial_collection"]
tutorial_collection.insert_many(random_collection.find())

InsertManyResult([ObjectId('662a575833b7e98b2b1488b5')], acknowledged=True)

In [ ]:
# 10 : 
# Inner join returns only the matching documents from both collections whereas outer join returns all documents 
# from both collections, including non-matching documents as null. The query seen during the course was an inner join.

In [ ]:
#Real World Problems

In [ ]:
#11: Use the oaipmh and api code get papers after January 2020 and for "cs,math,econ" categories. Insert them in MongoDB. Import only the first 200. 
#How is it sorted ? How can you define your own sort()? Query papers to get papers after 2021, which have 3 authors and with domain "cs".

import requests
import feedparser
import tqdm
import time
import pymongo

# For each id get all the metadata https://info.arxiv.org/help/api/basics.html#python_simple_example

client = pymongo.MongoClient('localhost',27017)
mydb = client["tutorial"]
collection = mydb["arxiv_api"]

# get list of ids previously downloaded
with open("C:/Users/Georgiana/Downloads/arxiv_cs.txt","r") as lines:
    ids = list(set(lines.read().split("\n")[0:-2]))

#init list of ids and iteration
ids_query = []

# loop through ids
for id_ in tqdm.tqdm(ids):
    #append id to list
    ids_query.append(id_)
    # if len list = 100 
    if len(ids_query) == 100 :
        # collapse list of id
        ids_query = ",".join(ids_query)
        # query the api for the 100 ids
        response = requests.get('http://export.arxiv.org/api/query?id_list={}&max_results=100'.format(ids_query))
        # parse response
        feed = feedparser.parse(response.content)
        # commit the 100 papers found
        list_of_insertion = []
        for entry in feed.entries:
            list_of_insertion.append(dict(entry))
        collection.insert_many(list_of_insertion)
        ids_query = []
        time.sleep(1/3)

In [ ]:
papers = collection.find({
    "published_parsed": {"$gte": "2021-01-01T00:00:00Z"},
    "$where": "this.authors.length === 3",
    "arxiv_primary_category.term": "cs"
})

for paper in papers:
    print(paper)

In [ ]:
#12: Do the same as exercise 8 but with the connection to the cluster. Then check the metrics and take screenshot of opcounters, logical size and connections.
collection.find( { "id": "http://arxiv.org/abs/2401.01379v1"} ).explain()['executionStats']


In [ ]:
collection.create_index([("id", 1)])
collection.find( { "id": "http://arxiv.org/abs/2401.01379v1"} ).explain()['executionStats']

In [ ]:
print("Opcounters:", client.admin.command("serverStatus")["opcounters"])
print("Taille logique:", client.admin.command("collstats", "arxiv_api")["size"])
print("Connexions:", client.admin.command("serverStatus")["connections"])

In [ ]:
pip install PIL

In [ ]:
#13 : Download a random image and store it in a collection.
from PIL import Image
# Using matplotlib to show the image
from matplotlib import pyplot
image = Image.open('primedelhumour.png')
pyplot.imshow(image)
pyplot.show()

In [ ]:
#14: Try to store a pandas dataframe in mongoDB (array with rownames, array with colnames and matrix with values)
import pandas as pd
import pymongo



client = pymongo.MongoClient('localhost', 27017)
mydb = client["TODO"]
collection = mydb["frompandas"]
# Create a pandas DataFrame
data = {
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
}
df = pd.DataFrame(data)


# Convert DataFrame to dictionary
df_dict = df.to_dict(orient='split')

# Insert the dictionary into MongoDB
collection.insert_one(df_dict)

In [ ]:
# 15: Insert the movie_review.tsv data into mongodb. Then query it to find the number of review that are positive and negative review. Fetch the docs which have "unexpected" in their review, how many are they ? Think of a clever way to count the number of words in the review using MongoDB (hint: Transform the review text before the insert in MongoDB) and create a density of number of words per review.
df=pd.read_csv("movie_review.tsv", sep="\t")
df.head()

In [ ]:
import re
db = client['TODO']  
collection = db['movies'] 

# Convertir la colonne "Phrase" en minuscules et supprimer la ponctuation
df['Phrase'] = df['Phrase'].str.lower()
df['Phrase'] = df['Phrase'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# Insérer les données dans MongoDB
data = df.to_dict(orient='records')
collection.insert_many(data)

# Compter le nombre d'avis positifs et négatifs
positive_reviews_count = collection.count_documents({"Sentiment": 1})
negative_reviews_count = collection.count_documents({"Sentiment": 2})

print("Nombre d'avis positifs:", positive_reviews_count)
print("Nombre d'avis négatifs:", negative_reviews_count)

# Récupérer les documents contenant le mot "unexpected" dans leur avis
unexpected_reviews = collection.find({"Phrase": {"$regex": "unexpected"}})
unexpected_reviews_count = collection.count_documents({"Phrase": {"$regex": "unexpected"}})

print("Nombre d'avis contenant 'unexpected':", unexpected_reviews_count)

pipeline = [
    {"$project": {"_id": 0, "Phrase": 1}},  # Sélectionner uniquement la colonne "Phrase"
    {"$addFields": {"word_count": {"$size": {"$split": ["$Phrase", " "]}}}},  # Ajouter une nouvelle colonne "word_count" contenant le nombre de mots dans chaque phrase
    {"$group": {"_id": None, "total_word_count": {"$sum": "$word_count"}, "total_reviews": {"$sum": 1}}},  # Calculer le nombre total de mots et le nombre total d'avis
    {"$project": {"_id": 0, "word_density": {"$divide": ["$total_word_count", "$total_reviews"]}}}  # Calculer la densité de mots par avis en divisant le nombre total de mots par le nombre total d'avis
]

result = list(collection.aggregate(pipeline))

if result:
    word_density = result[0]['word_density']
    print("Densité de mots par avis:", word_density)
else:
    print("Aucun document trouvé dans la collection.")

In [ ]:
# Real World Application 1 

In [ ]:
# Create an index, explain your choice of key.

import pymongo
import json

client = pymongo.MongoClient('localhost', 27017)
db = client["TODO"]
collection = db["pubmed"]

In [ ]:
with open('pubmed_cleaned.json', "r") as f:
    data = json.load(f)

for doc in data:
    if '_id' in doc:
        doc['_id'] = str(doc['_id']['$oid'])

try:
    collection.insert_many(data)
except pymongo.errors.BulkWriteError as e:
    print("Bulk write error occurred:", e)

In [ ]:
''' Je choisit le key '_id' pour l'index car c'est une valeur unique pour chaque article et il est 
utilisé pour identifier les articles dans la base de données pubmed.'''

collection.create_index([('_id',1)])

In [ ]:
# 2) Delete every paper that was published prior 2019 
from datetime import datetime
docs = collection.find()
for doc in docs:
    date_string = doc['date']
    date_string = date_string.replace("year", "").replace("month", "").replace("day", "").replace("hour", "").replace("minute", "").replace(",","-").replace(" ","")
    date = datetime.strptime(date_string, "%Y-%m-%d-%H-%M")
    if date.year < 2019:
        collection.delete_one({'_id':doc['_id']})

In [ ]:
# 3) How many paper have a single author ? Two authors ?
single_author_count = collection.count_documents({"$expr": {"$eq": [{"$size": {"$split": ["$authors", "\n"]},}, 1]}})
two_authors_count = collection.count_documents({"$expr": {"$eq": [{"$size": {"$split": ["$authors", "\n"]},}, 2]}})
print("Number of papers with a single author: ", single_author_count)
print("Number of papers with two authors: ", two_authors_count)

In [ ]:
# 4) What's the last paper inserted in the db ?
last_paper = collection.find().sort([("_id", -1)]).limit(1)
print("Last paper inserted in the db: ", last_paper[0]['pmid'], last_paper[0]['title'])

In [ ]:
# 5) Find articles with null meshwords.
docs = collection.find({"meshwords": None})
for doc in docs:
    print(doc['pmid'], doc['title'])

In [ ]:
# 6) Choose a keyword you are interested in (machine learning, computer vision,...). 
# Find the number of articles with the choosen keyword in their meshwords, abstract or title.
keyword = "machine learning"
docs = collection.find({"$or": [{"meshwords": {"$regex": keyword, "$options": "i"}},
                                 {"abstract": {"$regex": keyword, "$options": "i"}}, 
                                 {"title": {"$regex": keyword, "$options": "i"}}]})
doc_list = list(docs)
doc_num = len(doc_list)
print(doc_num)

In [ ]:
# 7) What's the number of articles that have at least one affiliation AND meshwords.
docs = collection.find({"$and": [{"affiliation": {"$ne": ""}}, 
                                 {"meshwords": {"$ne": ""}}]
                                 })
doc_list = list(docs)
doc_num = len(doc_list)
print("Le nombre d'article qui ont au moins une affiliation et des mots-clés est :", doc_num)

In [ ]:
#8. How many articles have a publishing date after 2020 ?

docs = collection.find({"date": {"$gt": datetime(2020, 1, 1)}})
doc_count = len(list(docs))
print(doc_count, "article on été publié en 2020")

In [ ]:
#9. Find articles where there's atleast one affiliation from a choosen country (you decide which one).
country = "france"
docs = collection.find({"authors": {"$regex": country, "$options": "i"}})
doc_list = list(docs)
doc_num = len(doc_list)
print("Nombre d'articles avec au moins une affiliation en France:", doc_num)

for doc in doc_list:
    print(doc['title'])


In [ ]:
#10. Check for any duplicates. (hint: look at the doi or the pmid)
pipline = [{"$group": {"_id": {"doi": "$doi"}, "count": {"$sum": 1}}}, 
           {"$match": {"count": {"$gt": 1}}}]

docs = collection.aggregate(pipline)
for doc in docs:
    print(doc)


In [ ]:
#11. Remove every articles where the abstract starts with an "R".
result = collection.delete_many({"abstract": {'$regex': "^R", "$options": "i"}})
deleted_count = result.deleted_count
print("Nombre d'articles supprimés:", deleted_count)

In [ ]:
# 12) Return the list of papers (pmid) where there's atleast one affiliation per author
docs = collection.find({})

pmid_list = []

for doc in docs:
    if 'authors' in doc:
        has_affiliation = False
        for author_line in doc['authors'].split('\n'):  # Split each line of authors
            if 'affil str' in author_line:  # Check if 'affil str' exists in the author line
                has_affiliation = True
                break  # Stop checking further if an affiliation is found for this document
        if has_affiliation:
            pmid_list.append(doc['pmid'])

print("Liste des articles où chaque auteur a au moins une affiliation:", pmid_list)





In [ ]:
#13. Create 500 random samples of the dataset, compute a statistics that you are interested in and check how it behaves through the different samples
import random

num_samples = 500
sample_size = 100
sample_statistics = []


def compute_statistic(sample_data):
    return sum(sample_data) / len(sample_data)

for _ in range(num_samples):
    docs = collection.find({})
    random_sample = random.sample(list(docs), sample_size)
    
    sample_data = [doc["nb_country"] for doc in random_sample]
    statistic = compute_statistic(sample_data)
    sample_statistics.append(statistic)

# Pour une meilleur vu dans l'ensemble on va faire un histogramme.
import matplotlib.pyplot as plt

plt.hist(sample_statistics, bins=20, edgecolor='black')
plt.xlabel('Valeurs de la statistique')
plt.ylabel('cumule du noumbre de pays')
plt.title('Distribution du nombre de pays à travers les échantillons aléatoires')
plt.show()


In [ ]:
# 14) Sandbox exercise: think of a problematic and try to answer it.


In [ ]:
#Real World Application 2 

In [ ]:
import pymongo
import json

client = pymongo.MongoClient('localhost', 27017)
db = client["Homeworks"]
collection = db["authors"]

In [ ]:
with open('authors.json', "r") as f:
    data = json.load(f)

for doc in data:
    if '_id' in doc:
        doc['_id'] = str(doc['_id']['$oid'])

collection.insert_many(data)

In [ ]:
# 1. Create an index, explain your choice of key.
''' _id est une valeur unique pour chaque document, c'est plus facile de faire les recherches par _id.'''

collection.create_index([("_id",1) ])

In [ ]:
# 2. What is the average length of "pmid_list"
pipeline = [
    {"$unwind": "$pmid_list"},
    {"$group": {"_id": None, "avg_length": 
                {"$avg": 
                 {"$strLenCP": 
                  {"$toString": "$pmid_list"}}}}}] # d'abord on transforme pmid_list en string et après on calcule la longueur

result = list(collection.aggregate(pipeline))
print(result[0]['avg_length'])

In [ ]:
# 3. How many distinct affiliations are there ?
pipeline = [
    {"$unwind": "$oa04_affiliations"}, 
    {"$group": {"_id": "$oa04_affiliations.Affiliation"}},  # group by affiliation
    {"$count": "num_unique_affiliations"} 
]

result = list(collection.aggregate(pipeline))
print(result)

In [ ]:
# 4.Find authors with atleast one "COM" AffiliationType

pipeline = [{"$unwind": "$oa04_affiliations"},
            {"$match": {"oa04_affiliations.AffiliationType": "COM"}},
            {"$project": {"_id": 1, "name": 1, "oa04_affiliations": 1}}]

result = list(collection.aggregate(pipeline))

authors_com = []
for author in result:
    authors_com.append(author['_id'])

In [ ]:
# 5.How many authors switched the AffiliationType ?

pipeline = [
    {"$unwind": "$oa04_affiliations"},
    {"$group": {"_id": "$_id", "num_types": {"$addToSet": "$oa04_affiliations.AffiliationType"}}},
    {"$project": {"_id": 1, "num_types": {"$size": "$num_types"}}},
    {"$match": {"num_types": {"$gt": 1}}},
    {"$count": "num_authors"}]

result = list(collection.aggregate(pipeline))
print(result)

In [ ]:
# 6. Find affiliation with the word "China"

pipeline = [{"$unwind": "$oa04_affiliations"},
    {"$match": {"oa04_affiliations.Affiliation": {"$regex": "China"}}},
    {"$project": {"_id": 1, "name": 1, "oa04_affiliations.Affiliation": 1}}]

result = list(collection.aggregate(pipeline))

In [ ]:
# 7. Get the pmids of papers published in 2019

pipeline = [
    {"$unwind": "$more_info"},
    {"$match": {"more_info.PubYear": 2019}},
    {"$project": {"_id": 1, "name": 1, "more_info.PMID": 1}}]

result = list(collection.aggregate(pipeline))
result[0] # exemple de résultat

In [ ]:
# 8. Count the number of doc with "oa06_researcher_education" OR "oa04_affiliations" key and 
# with the "oa06_researcher_education" AND "oa04_affiliations" .

pipeline1 = [{"$match": {"$or": [{"oa06_researcher_education": {"$exists": True}}, 
                                {"oa04_affiliations": {"$exists": True}}]}}]

pipeline2 =[{"$match": {"$and": [{"oa06_researcher_education": {"$exists": True}},
                                {"oa04_affiliations": {"$exists": True}}]}}]

                               
result1= list(collection.aggregate(pipeline1))
print(len(result1))

result2= list(collection.aggregate(pipeline2))
print(len(result2))

In [ ]:
# 9. What's the average "BeginYear", "BeginYear" is the type string, of "oa06_researcher_education".
pipeline = [{"$unwind": "$oa06_researcher_education"},
    {"$addFields": {"BeginYear": {"$cond": {"if": {"$eq": ["$oa06_researcher_education.BeginYear",' ']}, 
                                            "then":None, "else": {"$toInt": "$oa06_researcher_education.BeginYear"}}}}},
    {"$match": {"BeginYear": {"$ne":None}}},
    {"$group": {"_id": None, "avg_BeginYear": {"$avg": "$BeginYear"}}}]

result = list(collection.aggregate(pipeline))
print(result[0]['avg_BeginYear'])


In [ ]:
# 10.Count the distinct country of "oa06_researcher_education"

pipeline = [{"$unwind": "$oa06_researcher_education"},
    {"$group": {"_id": "$oa06_researcher_education.Country"}},
    {"$count": "num_unique_countries"}]

result = list(collection.aggregate(pipeline))
print(result)